In [41]:
import torch
import wandb

from default_mnist_config import create_default_mnist_config
from diffusion import DiffusionRunner
from models.classifier import ResNet, ResidualBlock, ConditionalResNet
from data_generator import DataGenerator

from tqdm.auto import trange

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [34]:
device = torch.device('cuda')
classifier_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
model = ResNet(**classifier_args)
model.to(device)

optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_func = torch.nn.CrossEntropyLoss()

In [35]:
datagen = DataGenerator(create_default_mnist_config())
train_generator = datagen.sample_train()

In [36]:
TOTAL_ITERS = 2_000
EVAL_FREQ = 500

### Обучите классификатор только на чистых картинках. Он понадобится нам для классификации условно сгенерированных картинок

In [39]:
model.train()

for iter_idx in trange(1, 1 + TOTAL_ITERS):
    
    """
    train
    """
    batch = next(train_generator)
    optim.zero_grad()
    outputs = model(batch[0].to(device))
    loss = loss_func(outputs, batch[1].to(device))
    loss.backward()
    optim.step()
    
    if iter_idx % EVAL_FREQ == 0:
        valid_accuracy, total = 0, 0
        """
        validate
        """
        model.eval()
        with torch.no_grad():
            for batch in datagen.valid_loader:
                outputs = model(batch[0].to(device)).cpu()
                _, predicted = torch.max(outputs.data, 1)
                valid_accuracy += (predicted==batch[1]).sum()
                total += len(batch[1])
        print('Clean MNIST classifier\'s accuracy:', valid_accuracy / total)
        model.train()
model.eval()

  0%|          | 0/2000 [00:00<?, ?it/s]

Clean MNIST classifier's accuracy: tensor(0.9945)
Clean MNIST classifier's accuracy: tensor(0.9937)
Clean MNIST classifier's accuracy: tensor(0.9923)
Clean MNIST classifier's accuracy: tensor(0.9939)


ResNet(
  (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [40]:
torch.save(model.state_dict(), './ddpm_checkpoints/clean_classifier.pth')